In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
base = '/users/jmy/data/image_sound'
sbjs_num = 15
sbj_list = [25,26,29,30,31,32,33,34,37,38,39,40,41,43,44]
affine = np.array([[   3. ,   -0. ,   -0. ,  -94.5],
       [  -0. ,    3. ,   -0. , -130.5],
       [   0. ,    0. ,    3. ,  -76.5],
       [   0. ,    0. ,    0. ,    1. ]])

In [ ]:
# Load CNN RDM
cdd_n, epoc = 5, 56
depth = 15
cnnrdms = np.zeros((sbjs_num,depth,16,16))
for li in range(depth):
    for sbj_idx in range(sbjs_num):
        filename = '_ind192to16_90_P{}_cdd0{}_L{}.npz'.format(str(sbj_list[sbj_idx]),str(cdd_n),str(li+1).zfill(2))
        savepath = base + '/RDMs/CNN/PCA/16x16/' + filename
        cnnrdms[sbj_idx,li] = np.load(savepath)['rdm']

In [ ]:
def rsa(stim_type,cnnrdm):
    iu = np.triu_indices(16,1)
    rho, p = stats.spearmanr(roirdm[iu],cnnrdm[iu])
    return rho, p 

def save_nifti(vol, savepath):
    img1 = nib.Nifti1Image(vol,affine=affine)
    nib.save(img1,savepath)

# Radius 3

In [ ]:
sv_key= 'PCA3_90'
savebase = base+'/RSA_Wholebrain_CNN/'+sv_key
# os.makedirs(savebase)

In [ ]:
import time
for sbj_idx in range(15): #25,39
    start_time = time.time()
    # load brain rdm
    load_path = base+'/RDMs/Neural/16x16/P'+str(sbj_list[sbj_idx])+'_i.npz' # r3
    l = np.load(load_path,allow_pickle=True)
    brain_rdm = l['rdm']
    brain_inform_list = l['info_list']
    nvv = brain_rdm.shape[0]
    type_, stim_type = 'Vs', 'etr'
    for li in range(15):
        vol_rho = np.zeros((64,76,64))
        vol_p = np.zeros((64,76,64))
        for vi in range(nvv):
            idx = brain_inform_list[vi][1]
            roirdm = brain_rdm[vi]
            vol_rho[idx], vol_p[idx] = rsa(stim_type,cnnrdms[sbj_idx,li])
        save_nifti(vol_rho,savebase+'/P'+str(sbj_list[sbj_idx])+'_rho_'+type_+'_'+stim_type+'_L'+str(li+1)+'.nii')
        save_nifti(vol_p,savebase+'/P'+str(sbj_list[sbj_idx])+'_pvl_'+type_+'_'+stim_type+'_L'+str(li+1)+'.nii')

In [ ]:
# grp layer map
arr_all = np.zeros((sbjs_num,depth,64,76,64))
for sbj_idx in range(sbjs_num):
    type_, stim_type = 'Vs', 'etr'
    for li in range(15):
        arr_all[sbj_idx,li] = nib.load(savebase+'/P'+str(sbj_list[sbj_idx])+'_rho_'+type_+'_'+stim_type+'_L'+str(li+1)+'.nii').get_fdata()

In [ ]:
# mask 
intsec_masks = np.zeros((depth,64,76,64))
for li in range(15):
    vol_sum = np.sum((arr_all[:,li]!=0),axis=0)
    its_idx = np.where(vol_sum == sbjs_num)
    intsec_masks[li][its_idx] = 1

In [ ]:
def vec2vol(vec,msk_idx):
    vol = np.zeros((64,76,64))
    vol[msk_idx] = vec
    return vol

In [ ]:
os.makedirs(savebase+'/Group')

In [ ]:
for li in range(depth):
    msk_idx = np.where(intsec_masks[li]==1)
    nvv = msk_idx[0].shape[0]
    tmparr = arr_all[:,li]
    # t-test for significance test
    #r to z
    vecs = np.arctanh(tmparr.transpose(1,2,3,0)[msk_idx])
    # t-test
    tmpt = np.zeros((nvv))
    tmpp = np.zeros((nvv))
    for vi, vec in enumerate(vecs):
        tmpt[vi], tmpp[vi] = stats.ttest_1samp(vec,0.)
    # save tmap, pmap
    save_nifti(vec2vol(tmpt,msk_idx),savebase+'/Group/tmap_'+type_+'_'+stim_type+'_L'+str(li+1)+'.nii')
    save_nifti(vec2vol(tmpp,msk_idx),savebase+'/Group/t-pmap_'+type_+'_'+stim_type+'_L'+str(li+1)+'.nii')

# p-value correction

In [ ]:
import os
import scipy
from tqdm import tqdm
from multiprocessing import Pool, Lock

In [ ]:
sbj_idx_ = np.arange(sbjs_num)

In [ ]:
# using multi core
p_cnt = 64 # number of cores 

In [ ]:
for li in range(depth):
    msk_idx = np.where(intsec_masks[li]==1)
    nvv = msk_idx[0].shape[0]
    run_cnt = int(nvv*5000/p_cnt)
    idx2shf = np.arange(nvv)
    tmparr = arr_all[:,li]
    # t-test for significance test
    #r to z
    vecs = np.arctanh(tmparr.transpose(1,2,3,0)[msk_idx])
    p_out = []
    boot_ts = np.zeros((1))
    def ttest_Rndperm(self):
        global sbj_idx_, vecs, boot_ts
        scipy.random.seed(int.from_bytes(os.urandom(3), byteorder='little'))
        rnd_idx = np.random.choice(idx2shf,sbjs_num,replace=False)
        boot_ts, _ = stats.ttest_1samp(vecs[rnd_idx,sbj_idx_],0.)
        return boot_ts
    if __name__=='__main__':
        p = Pool(processes = p_cnt) # 25
        for i in tqdm(range(run_cnt), desc='Total Progress') : # 400
            p_out.append(p.map(ttest_Rndperm, range(p_cnt))) # 25
        p.close()
        p.join()
    pooled_ts = np.array(p_out,dtype=np.float16).reshape(p_cnt*run_cnt)
    sns.distplot(pooled_ts)
    plt.title(type_)
    plt.show()
    # save
    np.savez(savebase+'/Group/rndperm_tscore_'+type_+'L'+str(li)+'_r3.npz',pooled_ts=pooled_ts)